def get_input_sizes(self, config):

        return [1, 1, 1, 1, 1, 1, 6, 6, 1, 1] 
        
for the values 6 is for R_arm, L_arm etc., 6 is for u_1:6, initial conditions for x, angular frequency, time  

We need

* Six individual parameters (R_arm, L_arm, etc.)

* Two arrays of size 6 (for u_1:6, initial conditions for x_1:6)

* One angular frequency (omega)

* One time (T)

In [3]:
import umbridge
import numpy as np

# Initialize the model
model = umbridge.HTTPModel("http://localhost:4242", "forward")

# Define the input parameters
R_arm = [0.5]     # Example value for R_arm
R_DC = [0.1]      # Example value for R_DC
R_AC = [0.2]      # Example value for R_AC
L_arm = [0.3]     # Example value for L_arm
L_DC = [0.4]      # Example value for L_DC
L_AC = [0.5]      # Example value for L_AC
u_parameters = [1, 2, 3, 4, 5, 6]  # Example values for u_1 to u_6 (size 6)
x_initial = [0, 0, 0, 0, 0, 0]      # Example initial conditions for x (size 6)
omega = [2*np.pi*50]                  # Example angular frequency
resolution = 125e-6
T = [0.02]   
T_eval = [t for t in np.arange(0, T[0] + resolution, resolution)]


# Create the parameters list
parameters = [R_arm, R_DC, R_AC, L_arm, L_DC, L_AC, u_parameters, x_initial, omega, T_eval]


# Call the model
result = model(parameters)
x_1 = np.array(result[0])
x_2 = np.array(result[1])
x_3 = np.array(result[2])
x_4 = np.array(result[3])
x_5 = np.array(result[4])
x_6 = np.array(result[5])
print(type(result))
print(type(u_parameters))

<class 'list'>
<class 'list'>


The following code is used for solving the 2nd equation (3b in the paper), so that we get the solutions for $x_7,x_8,x_9,x_{10},x_{11},x_{12}$

In [4]:
from scipy.integrate import odeint

# T matrix 
matrix_T = np.array( [[1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6)],
                      [-1/np.sqrt(6), -1/np.sqrt(6), -1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6)],
                      [1/2, -1/2, 0, -1/2, 1/2, 0],
                      [1/np.sqrt(12), 1/np.sqrt(12), -1/np.sqrt(3), -1/np.sqrt(12), -1/np.sqrt(12), 1/np.sqrt(3)],
                      [-1/2, 1/2, 0, -1/2, 1/2, 0],
                      [-1/np.sqrt(12), -1/np.sqrt(12), 1/np.sqrt(3), -1/np.sqrt(12), -1/np.sqrt(12), 1/np.sqrt(3)],
                     ])



T_transpose = np.transpose(matrix_T)

deriv_x_7_12 = (T_transpose @ u_parameters)   *  (T_transpose @ result)

def model(x, t):
    dx_dt = deriv_x_7_12  # The right-hand side of the equation is constant
    return dx_dt

# Initial condition
x0 = np.zeros(6)  # Assuming initial condition x(0) = [0, 0, 0, 0, 0, 0]

# Solve ODE
solution_x_7_12 = odeint(model, y0, T_eval)
x_7 = solution_x_7_12[:, 0]
x_8 = solution_x_7_12[:, 1]
x_9 = solution_x_7_12[:, 2]
x_10 = solution_x_7_12[:, 3]
x_11 = solution_x_7_12[:, 4]
x_12 = solution_x_7_12[:, 5]

ValueError: operands could not be broadcast together with shapes (6,) (6,161) 

In [9]:
print(T_transpose @ u_parameters)


[-1.98559856  0.01440144  0.74645225 -5.66200647  2.33799353  6.99824756]


In [10]:
print(T_transpose @ result)

[[ 0.00000000e+00 -6.31060542e-04 -1.26193044e-03 -1.89260973e-03
  -2.52309846e-03 -3.15339668e-03 -3.78350444e-03 -4.41342177e-03
  -5.04314872e-03 -5.67268533e-03 -6.30203166e-03 -6.93118773e-03
  -7.56015361e-03 -8.18892933e-03 -8.81751494e-03 -9.44591048e-03
  -1.00741160e-02 -1.07021315e-02 -1.13299571e-02 -1.19575929e-02
  -1.25850387e-02 -1.32122948e-02 -1.38393611e-02 -1.44662377e-02
  -1.50929247e-02 -1.57194220e-02 -1.63457297e-02 -1.69718479e-02
  -1.75977766e-02 -1.82235159e-02 -1.88490658e-02 -1.94744263e-02
  -2.00995975e-02 -2.07245794e-02 -2.13493721e-02 -2.19739756e-02
  -2.25983900e-02 -2.32226153e-02 -2.38466516e-02 -2.44704989e-02
  -2.50941571e-02 -2.57176265e-02 -2.63409071e-02 -2.69639987e-02
  -2.75869017e-02 -2.82096158e-02 -2.88321413e-02 -2.94544781e-02
  -3.00766264e-02 -3.06985861e-02 -3.13203572e-02 -3.19419399e-02
  -3.25633342e-02 -3.31845400e-02 -3.38055576e-02 -3.44263868e-02
  -3.50470278e-02 -3.56674806e-02 -3.62877452e-02 -3.69078217e-02
  -3.75277

This is for the last equation $\tilde{y} = \tilde{C}x_{1:6}$. To find the initial conditions for $x_{1:6}$, we are using the function of $y_{ref}$ at time $t=0$. Then we have the following linear system 

\begin{equation}
\left(\begin{array}{cc} 
y_{1,ref}(0) \\
y_{2,ref}(0) \\
y_{3,ref}(0) \\
\end{array}\right)
= - \left(\begin{array}{c} 
1 & 0 & 0 & 1 & 0 & 0 \\
0 & 1 & 0 & 0 & 1 & 0 \\
0 & 0 & 1 & 0 & 0 & 1 \\
\end{array}\right) 
\left(\begin{array}{c} 
x_1(0) \\
x_2(0) \\
x_3(0) \\
x_4(0) \\
x_5(0) \\
x_6(0) \\
\end{array}\right)  
\end{equation}

Since we dont have square matrix, the equation is under or over determined. By using the least square method, we can estimate the solution.

In [11]:
y_ref = [0] * 3
I_AC = 15
phi_AC = 0
matrix_C = np.array([[1, 0, 0, 1, 0, 0],
              [0, 1, 0, 0, 1, 0],
              [0, 0, 1, 0, 0, 1]])

#print(np.linalg.inv(matrix_C))


def y(T, omega):
        for k in range(1, 4):
            y_ref[k-1]= I_AC * np.cos (omega * T - (2 * np.pi * (k-1))/3 -phi_AC)

        return y_ref

y0_values = y(0, 50)
x0, residuals, rank, s = np.linalg.lstsq(matrix_C, y0_values, rcond=None)
print("initial condition values for x_1 to x_6 using least squares:", x0)

initial condition values for x_1 to x_6 using least squares: [ 7.5  -3.75 -3.75  7.5  -3.75 -3.75]


We need to have some initial conditions for $x_7$ to $x_{12}$. We can do the same as above. 

In [ ]:
resolution = 125e-6
T = [0.02]   
T_eval = [t for t in np.arange(0, T[0] + resolution, resolution)]
print(T_eval)